In [7]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'r8BGSocOtv6vZBnjcwkU5qM4u'
consumer_secret = 'CNAJg23IWXLvUUyJfuFUQOYfzvtKfQVQu6SsY82yJY7GGdhBDC'
access_token = '796083347315367936-grZPgV2cW8DvrozevvihPIJ1Oq9WQVK'
access_secret = 'oJoGFZYDDOXjyG30v0FBChWmGfF9RG6fXvDCVuNrDZJGb'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [8]:
def process_or_store(tweet):
    print(json.dumps(tweet))

In [9]:
import json
for tweet in tweepy.Cursor(api.user_timeline).items():
    #print(tweet._json)
    process_or_store(tweet._json)

{"place": null, "contributors": null, "lang": "en", "is_quote_status": false, "favorited": false, "in_reply_to_user_id_str": null, "entities": {"hashtags": [], "urls": [], "user_mentions": [], "symbols": []}, "geo": null, "favorite_count": 0, "in_reply_to_user_id": null, "in_reply_to_status_id_str": null, "created_at": "Tue Nov 08 21:44:27 +0000 2016", "text": "hey hey", "in_reply_to_status_id": null, "retweeted": false, "id_str": "796106130590203904", "id": 796106130590203904, "user": {"is_translation_enabled": false, "is_translator": false, "profile_text_color": "333333", "has_extended_profile": false, "profile_sidebar_fill_color": "DDEEF6", "verified": false, "following": false, "name": "Irina Stolbova", "default_profile_image": true, "url": null, "screen_name": "stolbova_ir", "favourites_count": 0, "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_6_normal.png", "entities": {"description": {"urls": []}}, "utc_offset": null, "followers_count": 

In [12]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('negative.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
twitter_stream = Stream(auth, MyListener())
#twitter_stream.filter(track=[':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(='], languages=['fr'])
twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')='], languages=['fr'])

RuntimeError: No active exception to reraise

In [2]:
from nltk.tokenize import word_tokenize
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(word_tokenize(tweet))

['RT', '@', 'marcobonzanini', ':', 'just', 'an', 'example', '!', ':', 'D', 'http', ':', '//example.com', '#', 'NLP']


In [22]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [23]:
tweet = "RT @marcobonzanini: just I'd like to see an example! :D http://example.com #NLP"
print(preprocess(tweet))

['RT', '@marcobonzanini', ':', 'just', "I'd", 'like', 'to', 'see', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [40]:
from nltk.corpus import stopwords
import string

smiles_negativ = [':(', ':-(', '=(', ')-:', '):', ');', ')=']
smiles_positiv = [':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=']
smiles = smiles_negativ + smiles_positiv
punctuation = list(string.punctuation)
stop = stopwords.words('french') + punctuation + ['RT', 'la', 'le', 'les', 'des', '’', 'Le', 'La', 'Les', 'Des', 'é', 'À'] + smiles

In [41]:
import json
import operator
from nltk import bigrams
from collections import Counter

tweets_text = []
count_all = Counter()

for line in open('positive.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
  except:
    pass
#print(len(tweets_text))

for var_text in tweets_text:
    tweet_prep = preprocess(var_text)
    terms_all = [term for term in tweet_prep if term not in stop and not term.startswith(('@', '#', 'http'))]
    terms_bigram = bigrams(terms_all)
    count_all.update(terms_bigram)

print(count_all.most_common(30))

[(('Pour', 'participer'), 103), (('Follow', '…'), 94), (('participer', 'Follow'), 94), (('€', 'Pour'), 94), (('Concours', 'On'), 93), (('week-end', '2'), 89), (('passe', 'week-end'), 89), (('2', '679'), 89), (('On', 'passe'), 89), (('679', '€'), 89), (('Mur', 'Parisiens'), 49), (('Parisiens', '1'), 49), (('1', 'cadeau'), 49), (('Adidas', 'choix'), 49), (('cadeau', 'jour'), 49), (('26', 'Follow'), 49), (('choix', 'cadeau'), 49), (('Follow', 'gagner'), 49), (('cadeau', '…'), 26), (('19', '26'), 26), (('jour', '19'), 26), (('Maillot', 'Adidas'), 26), (('gagner', 'Maillot'), 26), (('jour', '18'), 23), (('gagner', 'Polo'), 23), (('18', '26'), 23), (('Polo', 'Adidas'), 23), (('cadeau', 'r'), 23), (('r', '…'), 23), (('vid', 'éo'), 18)]
